#### Rex Gayas
#### Week 7 Assignment 7.2 Spring 2024
#### DSC400-T301 Big Data, Technology, and Algo (2245-1)
#### Stream Processing with Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

# Initialize a SparkSession for structured streaming
spark = SparkSession \
    .builder \
    .appName("Assignment 7.1") \
    .getOrCreate()

# Define the input data stream coming from netcat
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words using the explode and split functions
words = lines.select(
    explode(
        split(lines.value, " ")
    ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

# Modify the query to only return results where the word count is greater than two
filteredWordCounts = wordCounts.filter(wordCounts['count'] > 2)

# Define the query to write the results to the console
try:
    query = filteredWordCounts \
        .writeStream \
        .outputMode("complete") \
        .format("console") \
        .start()

    query.awaitTermination()
except KeyboardInterrupt:
    print('Stopping query')
    query.stop()


24/04/28 16:45:46 WARN Utils: Your hostname, Rexsophy resolves to a loopback address: 127.0.1.1; using 172.26.141.55 instead (on interface eth0)
24/04/28 16:45:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/28 16:45:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/28 16:45:49 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/28 16:45:50 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/04/28 16:45:51 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7153c352-835d-4cbb-8d28-89b4cdebd446. If it's required to delete it under any circumstances, pleas

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|  hello|    4|
|science|    3|
|  spark|    4|
|  world|    3|
+-------+-----+



24/04/28 17:00:20 WARN TextSocketMicroBatchStream: Stream closed by localhost:9999


Modified the query to only return results where the word count is greater than two and this was executed by adding a filter to the DataFrame before writing the stream.